In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

from modules.info import *
from modules.save2csv import *


In [2]:
entity_with_chapter = pd.read_csv(path + 'outputs/4_章节触发结果_' + date + '.csv')
model_rule = pd.read_excel(path + 'rules/模型规则_230526.xlsx', sheet_name='章节_政策')

In [3]:
model_rule.rename(columns={'章节编码':'chapter_code',
                           '政策编码':'policy_code'}, inplace=True)
model_rule.drop(columns='逻辑关系', inplace=True)
model_rule.head()

,chapter_code,policy_code
0,CSRC-IPOLaw-2,CSRC-IPOLaw
1,SZSE-GEMIPOLaw-2,SZSE-GEMIPOLaw
2,SZSE-GEMIPOLaw-10,SZSE-GEMIPOLaw
3,SZSE-IPOLaw-3,SZSE-IPOLaw
4,SZSE-IPOLaw-9,SZSE-IPOLaw


In [4]:
entity_with_chapter_and_policy = pd.merge(entity_with_chapter, model_rule, on='chapter_code', how='left')
entity_with_chapter_and_policy.head()

,stock_code,stock_name,entity_name,year,chapter_code,chapter_result_danger,chapter_result_normal,policy_code
0,000002.SZ,万科A,万科企业股份有限公司,2021,BSE-IPOLaw-2,False,False,BSE-IPOLaw
1,000002.SZ,万科A,万科企业股份有限公司,2020,BSE-IPOLaw-2,False,False,BSE-IPOLaw
2,000002.SZ,万科A,万科企业股份有限公司,2019,BSE-IPOLaw-2,False,False,BSE-IPOLaw
3,000006.SZ,深振业A,深圳市振业(集团)股份有限公司,2021,BSE-IPOLaw-2,False,False,BSE-IPOLaw
4,000006.SZ,深振业A,深圳市振业(集团)股份有限公司,2020,BSE-IPOLaw-2,False,False,BSE-IPOLaw


In [5]:
# 进行政策触发判断
grouped = entity_with_chapter_and_policy.groupby(['stock_code', 'stock_name', 'entity_name', 'year', 'chapter_code'])
entity_with_chapter_and_policy['policy_result_danger'] = grouped['chapter_result_danger'].transform('any')
entity_with_chapter_and_policy['policy_result_normal'] = grouped['chapter_result_normal'].transform('any')

entity_with_chapter_and_policy.drop(columns=['chapter_code', 'chapter_result_danger', 'chapter_result_normal'], inplace=True)
entity_with_policy = entity_with_chapter_and_policy.drop_duplicates()
entity_with_policy.head()

,stock_code,stock_name,entity_name,year,policy_code,policy_result_danger,policy_result_normal
0,000002.SZ,万科A,万科企业股份有限公司,2021,BSE-IPOLaw,False,False
1,000002.SZ,万科A,万科企业股份有限公司,2020,BSE-IPOLaw,False,False
2,000002.SZ,万科A,万科企业股份有限公司,2019,BSE-IPOLaw,False,False
3,000006.SZ,深振业A,深圳市振业(集团)股份有限公司,2021,BSE-IPOLaw,False,False
4,000006.SZ,深振业A,深圳市振业(集团)股份有限公司,2020,BSE-IPOLaw,False,False


In [6]:
save2csv(entity_with_policy, '5_政策触发结果')